In [3]:
from google.cloud import bigquery
import pandas as pd
pd.set_option('display.width', 1000)

client = bigquery.Client(project="matjib-c9682")

/Users/swislar/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/swislar/anaconda3/lib/python3.10/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


# Q1


What are the key factors influencing customer purchasing behavior?

- Analyze historical sales data to identify patterns and trends.
- <strong>Develop customer segmentation models based on purchasing behavior. (Not done)</strong>


## Google Analytics


### Exit rate by page title


In [90]:
# Exit rate by pages
query = f"""
        SELECT hits.page.pageTitle as PageTitle, COUNT(*) AS Views, SUM(totals.bounces)/COUNT(*) AS ExitRate
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, UNNEST(hits) AS hits
        WHERE _TABLE_SUFFIX BETWEEN '20160901' AND '20160930' AND hits.type='PAGE'
        GROUP BY PageTitle
        ORDER BY Views DESC
        LIMIT 10
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

                               PageTitle  Views  ExitRate
0                                   Home  56486  0.183939
1                    Google Online Store  36264  0.509128
2                         Men's-T-Shirts  18320  0.060535
3                          Shopping Cart  17932  0.010930
4                                 Office  13069  0.013773
5                            Electronics  10269  0.011686
6                              Drinkware  10203  0.026169
7                                Apparel  10055  0.021681
8                                   Bags   9928  0.027699
9  The Google Merchandise Store - Log In   8636  0.013432 



Exit rates are the lowest when items from a specific category are being displayed


### Bounce rates by PageTitle


In [7]:
# Bounce rate by pages
query = f"""
        SELECT hits.page.pageTitle AS PageTitle, COUNT(*) AS Views, SUM(totals.bounces)/COUNT(*) AS BounceRate
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, UNNEST(hits) as hits
        WHERE _TABLE_SUFFIX BETWEEN '20160901' AND '20160930' AND hits.type = 'PAGE' AND hits.hitNumber = 1
        GROUP BY PageTitle
        ORDER BY Views DESC
        LIMIT 10
        """
rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

             PageTitle  Views  BounceRate
0  Google Online Store  31730    0.581878
1                 Home  26026    0.397641
2       Men's-T-Shirts   2852    0.388850
3     Page Unavailable    755    0.413245
4            Drinkware    667    0.398801
5                 Bags    643    0.427683
6              YouTube    560    0.512500
7      Men's-Outerwear    489    0.361963
8              Apparel    464    0.465517
9        Shopping Cart    464    0.422414 



# Q2


How can we improve customer retention and lifetime value?

- Calculate customer churn rates and identify at-risk customers.
- <strong>Analyze the effectiveness of current retention strategies. (Not done)</strong>


## Google Analytics


In [259]:
query = f"""
        SELECT
            fullVisitorID,
            channelGrouping AS Channel,
            SUM(totals.totalTransactionRevenue) AS TotalRevenue,
            COUNT(fullVisitorID) AS CustomerPurchaseCount,
            trafficSource.source AS TrafficSource,
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits,
            UNNEST(hits.product) AS product
        WHERE
            _TABLE_SUFFIX BETWEEN '20160901' AND '20160930' AND
            hits.eCommerceAction.action_type = '6'
        GROUP BY fullVisitorID, Channel, TrafficSource
        ORDER BY CustomerPurchaseCount DESC
        LIMIT 10
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

         fullVisitorID         Channel  TotalRevenue  CustomerPurchaseCount TrafficSource
0  8851822767477687842        Referral   30878760000                     96      (direct)
1   666582393118140484        Referral   20600080000                     68      (direct)
2  3033366073043100694        Referral   11512000000                     60      (direct)
3  5689475961693395998        Referral   15128780000                     54      (direct)
4  9377429831454005466  Organic Search   20728520000                     52        google
5  8436426603099391262        Referral   37197920000                     52      (direct)
6  5632276788326171571          Direct  673375500000                     42      (direct)
7  0780206376162514125        Referral   20200000000                     40      (direct)
8  4923243433046835289          Direct   14585600000                     40      (direct)
9  2854722262909538790  Organic Search    4431400000                     40        google 



The most popular channel that is used by customers with a high purchase count


In [81]:
# https://medium.com/octave-john-keells-group/a-simple-six-step-approach-to-define-customer-churn-in-retail-f401e31e57c0#:~:text=Identifying%20the%20point%20in%20which,time%20to%20proactively%20implement%20interventions.&text=Step%206%3A,is%20identified%20as%20a%20churner.
query = f"""
        -- Compute the Number of purchases made by each visitor in a day 
        -- NULL transaction means that there are no transactions made
        WITH DailyPurchases AS (
            SELECT
                PARSE_DATE('%Y%m%d', _TABLE_SUFFIX) AS Date,
                fullVisitorID,
                SUM(IF(totals.transactions IS NOT NULL, totals.transactions, 0)) AS DailyPurchaseCount
            FROM
                `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
                UNNEST(hits) AS hits
            GROUP BY
                Date,
                fullVisitorID
        ),
        -- Compute the 30 Day Moving Average and the 90 Day Moving Average STD
        MovingAverages AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                COALESCE(
                    SUM(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
                    )/30, 0
                ) AS OneMonthMA,
                COALESCE(
                    STDDEV(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
                    ) * (COUNT(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
                    ) - 1) / 89, 0
                ) AS ThreeMonthSTDMA
            FROM
                DailyPurchases
        ),
        -- Find the threshold at which the customer is considered at risk of retention
        Threshold AS (
            SELECT
                Date, 
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                (OneMonthMA - ThreeMonthSTDMA) AS LowerBound
            FROM 
                MovingAverages
        ),
        -- Lag the threshold by 30 Days and the visitor should not be considered at risk immediately
        LaggingThreshold AS (
            SELECT
                Date, 
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                LowerBound,
                COALESCE(
                    LAG(LowerBound, 29) OVER (PARTITION BY fullVisitorID ORDER BY Date), 0
                ) AS LowerBoundOneMonthLag
            FROM
                Threshold
            ORDER BY
                Date
        ),
        -- Select only the latest date for each visitor and check for their churn status
        RankedDates AS (
            SELECT
                Date, 
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                LowerBound,
                LowerBoundOneMonthLag,
                ROW_NUMBER() OVER (
                    PARTITION BY fullVisitorID
                    ORDER BY Date DESC
                ) AS rn
            FROM LaggingThreshold
        )
        
        SELECT Date, fullVisitorID, DailyPurchaseCount, OneMonthMA, LowerBound, LowerBoundOneMonthLag
        FROM RankedDates
        ORDER BY Date
        """

rows = client.query(query)

result_df = rows.to_dataframe()

grouped_sum = result_df.groupby('fullVisitorID')[
    'DailyPurchaseCount'].sum().reset_index()

grouped_sum = grouped_sum.rename(
    columns={'DailyPurchaseCount': 'HistoricPurchases'})

df = result_df.merge(grouped_sum, on='fullVisitorID',
                     how='left').query("HistoricPurchases > 0")

df['Date'] = pd.to_datetime(df['Date'])

latest_idx = df.groupby('fullVisitorID')['Date'].idxmax()

result_df = df.loc[latest_idx].sort_values(by="Date")

print(result_df, "\n")

             Date        fullVisitorID  DailyPurchaseCount  OneMonthMA  LowerBound  LowerBoundOneMonthLag  HistoricPurchases
408    2016-08-01  5760753352577829144                  79    2.633333    2.633333                    0.0                 79
717    2016-08-01  6569605994631186947                  36    1.200000    1.200000                    0.0                 36
80     2016-08-01  2125540555068339394                  25    0.833333    0.833333                    0.0                 25
432    2016-08-01  7589137567725941774                  51    1.700000    1.700000                    0.0                 51
756    2016-08-01  5563168194966233133                  21    0.700000    0.700000                    0.0                 21
...           ...                  ...                 ...         ...         ...                    ...                ...
834084 2017-08-01   024507252193437459                  16    0.533333    0.406213                    0.0                 16


/var/folders/lj/tfjxb4wd4kx1kwshmxnt57840000gn/T/ipykernel_33309/253609387.py:103: RuntimeWarning: Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.
  how='left').query("HistoricPurchases > 0")


In [82]:
# Those that have not made a purchase within the last 3 months or have a monthly moving average less than the lower bound have a retention risk
max_date = result_df['Date'].max()
three_months_ago = max_date - pd.DateOffset(months=3)
RetentionRiskCount = len(set(result_df.query(
    "Date <= @three_months_ago | LowerBoundOneMonthLag >= OneMonthMA").fullVisitorID))

# Those that have made a purchase within the last 3 months or have a monthly moving average more than the lower bound do not have a retention risk
RetentionCount = len(set(result_df.query(
    "@three_months_ago <= Date <= @max_date & LowerBoundOneMonthLag < OneMonthMA").fullVisitorID))


result_df.query(
    "@three_months_ago <= Date <= @max_date & LowerBoundOneMonthLag < OneMonthMA")

,Date,fullVisitorID,DailyPurchaseCount,OneMonthMA,LowerBound,LowerBoundOneMonthLag,HistoricPurchases
650611,2017-05-01,536428604379922267,22,0.733333,0.733333,0.0,22
651870,2017-05-01,1749853719127085935,13,0.433333,0.433333,0.0,13
650822,2017-05-01,9740050408987139130,19,0.633333,0.633333,0.0,19
650153,2017-05-01,9565722193439879038,11,0.366667,0.279271,0.0,11
650942,2017-05-01,4501448098783068461,22,2.466667,1.636650,0.0,74
...,...,...,...,...,...,...,...
834084,2017-08-01,024507252193437459,16,0.533333,0.406213,0.0,16
835321,2017-08-01,5199370466032130686,0,0.866667,0.428464,0.0,26
834618,2017-08-01,3614707430894059857,28,0.933333,0.933333,0.0,28
833186,2017-08-01,9591202457292182670,20,0.666667,0.157052,0.0,20


These are the at risk customers as the lagged lower bound is greater or equal to their one month moving average on purchase count


In [73]:
query = f"""
        -- Compute the Number of purchases made by each visitor in a day 
        -- NULL transaction means that there are no transactions made
        WITH DailyPurchases AS (
            SELECT
                PARSE_DATE('%Y%m%d', _TABLE_SUFFIX) AS Date,
                fullVisitorID,
                SUM(IF(totals.transactions IS NOT NULL, totals.transactions, 0)) AS DailyPurchaseCount
            FROM
                `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
                UNNEST(hits) AS hits
            GROUP BY
                Date,
                fullVisitorID
        ),
        -- Compute the 30 Day Moving Average and the 90 Day Moving Average STD
        MovingAverages AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                COALESCE(
                    SUM(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
                    )/30, 0
                ) AS OneMonthMA,
                COALESCE(
                    STDDEV(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
                    ) * (COUNT(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
                    ) - 1) / 89, 0
                ) AS ThreeMonthSTDMA
            FROM
                DailyPurchases
        ),
        -- Find the threshold at which the customer is considered at risk of retention
        Threshold AS (
            SELECT
                Date, 
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                (OneMonthMA - ThreeMonthSTDMA) AS LowerBound
            FROM 
                MovingAverages
        ),
        -- Lag the threshold by 30 Days and the visitor should not be considered at risk immediately
        LaggingThreshold AS (
            SELECT
                Date, 
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                LowerBound,
                COALESCE(
                    LAG(LowerBound, 29) OVER (PARTITION BY fullVisitorID ORDER BY Date), 0
                ) AS LowerBoundOneMonthLag
            FROM
                Threshold
            ORDER BY
                Date
        ),
        -- Find the channels used by those with a retention risk
        RetentionRisk AS (
            SELECT 
                channelGrouping AS Channel,
                SUM(IF(totals.transactions IS NOT NULL, totals.transactions, 0)) AS Transactions
            FROM 
                `bigquery-public-data.google_analytics_sample.ga_sessions_*` df
            RIGHT JOIN 
                LaggingThreshold lt
            ON 
                df.fullVisitorID = lt.fullVisitorID
            WHERE
                lt.LowerBoundOneMonthLag >= lt.OneMonthMA
            GROUP BY 
                Channel
        ),
        TotalTransactions AS (
            SELECT 
                SUM(Transactions) AS Total
            FROM 
                RetentionRisk
        )

        SELECT 
            Channel,
            Transactions,
            (Transactions / TotalTransactions.Total) * 100 AS Transactions_Pct
        FROM 
            RetentionRisk,
            TotalTransactions
        ORDER BY 
            Transactions_Pct DESC
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

          Channel  Transactions  Transactions_Pct
0        Referral          5563         45.676985
1  Organic Search          2796         22.957550
2         Display          1701         13.966664
3          Direct          1690         13.876345
4     Paid Search           323          2.652106
5          Social           102          0.837507
6      Affiliates             4          0.032843
7         (Other)             0          0.000000 



Overview of channels used by those who do not have a retention risk


In [74]:
# CHANGE THE TOP HALF FOR THIS
query = f"""
        -- Compute the Number of purchases made by each visitor in a day 
        -- NULL transaction means that there are no transactions made
        WITH DailyPurchases AS (
            SELECT
                PARSE_DATE('%Y%m%d', _TABLE_SUFFIX) AS Date,
                fullVisitorID,
                SUM(IF(totals.transactions IS NOT NULL, totals.transactions, 0)) AS DailyPurchaseCount
            FROM
                `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
                UNNEST(hits) AS hits
            GROUP BY
                Date,
                fullVisitorID
        ),
        -- Compute the 30 Day Moving Average and the 90 Day Moving Average STD
        MovingAverages AS (
            SELECT
                Date,
                fullVisitorID,
                DailyPurchaseCount,
                COALESCE(
                    SUM(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
                    )/30, 0
                ) AS OneMonthMA,
                COALESCE(
                    STDDEV(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
                    ) * (COUNT(DailyPurchaseCount) OVER (
                        PARTITION BY fullVisitorID
                        ORDER BY Date
                        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
                    ) - 1) / 89, 0
                ) AS ThreeMonthSTDMA
            FROM
                DailyPurchases
        ),
        -- Find the threshold at which the customer is considered at risk of retention
        Threshold AS (
            SELECT
                Date, 
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                (OneMonthMA - ThreeMonthSTDMA) AS LowerBound
            FROM 
                MovingAverages
        ),
        -- Lag the threshold by 30 Days and the visitor should not be considered at risk immediately
        LaggingThreshold AS (
            SELECT
                Date, 
                fullVisitorID,
                DailyPurchaseCount,
                OneMonthMA,
                LowerBound,
                COALESCE(
                    LAG(LowerBound, 29) OVER (PARTITION BY fullVisitorID ORDER BY Date), 0
                ) AS LowerBoundOneMonthLag
            FROM
                Threshold
            ORDER BY
                Date
        ),
        -- Find the channels used by those without a retention risk
        Retention AS (
            SELECT 
                channelGrouping AS Channel,
                SUM(IF(totals.transactions IS NOT NULL, totals.transactions, 0)) AS Transactions
            FROM 
                `bigquery-public-data.google_analytics_sample.ga_sessions_*` df
            RIGHT JOIN 
                LaggingThreshold lt
            ON 
                df.fullVisitorID = lt.fullVisitorID
            WHERE
                lt.LowerBoundOneMonthLag < lt.OneMonthMA
            GROUP BY 
                Channel
        ),
        TotalTransactions AS (
            SELECT 
                SUM(Transactions) AS Total
            FROM 
                Retention
        )

        SELECT 
            Channel,
            Transactions,
            (Transactions / TotalTransactions.Total) * 100 AS Transactions_Pct
        FROM 
            Retention,
            TotalTransactions
        ORDER BY 
            Transactions_Pct DESC
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

          Channel  Transactions  Transactions_Pct
0        Referral         14366         39.819280
1          Direct          9594         26.592383
2  Organic Search          8712         24.147680
3         Display          1903          5.274683
4     Paid Search          1203          3.334442
5          Social           285          0.789955
6      Affiliates            14          0.038805
7         (Other)             1          0.002772 



Overview of channels used by those who do not have a retention risk


In [83]:
ChurnRate = (RetentionRiskCount /
             (RetentionRiskCount + RetentionCount)) * 100

print("The churn rate for google online store is :", ChurnRate, "% \n")

The churn rate for google online store is : 69.27042030134814 % 



# Q3


What are the most effective marketing channels and campaigns? - Evaluate the ROI of different marketing channels.

- Analyze the impact of various promotional campaigns on sales.


## Google Analytics


In [94]:
query = f"""
        SELECT
            channelGrouping AS Channel,
            trafficSource.source AS TrafficSource,
            CASE 
                WHEN hits.eCommerceAction.action_type = '1' THEN 'Click product lists'
                WHEN hits.eCommerceAction.action_type = '2' THEN 'View product details'
                WHEN hits.eCommerceAction.action_type = '3' THEN 'Add to cart'
                WHEN hits.eCommerceAction.action_type = '4' THEN 'Remove from cart'
                WHEN hits.eCommerceAction.action_type = '5' THEN 'Check out'
                WHEN hits.eCommerceAction.action_type = '6' THEN 'Purchase'
                WHEN hits.eCommerceAction.action_type = '7' THEN 'Refund'
                WHEN hits.eCommerceAction.action_type = '0' THEN 'Unknown'
            END AS Action,
            COUNT(fullVisitorID) AS UserCount
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits,
            UNNEST(hits.product) AS product
        WHERE
            _TABLE_SUFFIX BETWEEN '20160901' AND '20160930' AND
            hits.eCommerceAction.action_type != '0'
        GROUP BY Action, TrafficSource, Channel
        ORDER BY UserCount DESC
        LIMIT 20
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

           Channel TrafficSource                Action  UserCount
0   Organic Search        google   Click product lists      19548
1   Organic Search        google  View product details      15955
2         Referral      (direct)   Click product lists      11542
3         Referral      (direct)  View product details       9415
4           Direct      (direct)   Click product lists       8885
5         Referral      (direct)             Check out       7928
6           Direct      (direct)  View product details       7156
7   Organic Search      (direct)   Click product lists       5877
8   Organic Search      (direct)  View product details       4741
9   Organic Search        google             Check out       4426
10  Organic Search        google           Add to cart       3659
11          Direct      (direct)             Check out       3426
12        Referral      (direct)           Add to cart       3104
13        Referral      (direct)              Purchase       2584
14        

Using the dataframe above, we can find which platform and method is the most effective in influencing sales


In [146]:
query = f"""
        SELECT
            channelGrouping AS Channel,
            SUM(totals.totalTransactionRevenue) AS TotalRevenue,
            COUNT(fullVisitorID) AS UserCount,
            SUM(totals.totalTransactionRevenue) / COUNT(fullVisitorID) AS AVGRevenuePerUser,
            trafficSource.source AS TrafficSource,
            CASE 
                WHEN hits.eCommerceAction.action_type = '1' THEN 'Click product lists'
                WHEN hits.eCommerceAction.action_type = '2' THEN 'View product details'
                WHEN hits.eCommerceAction.action_type = '3' THEN 'Add to cart'
                WHEN hits.eCommerceAction.action_type = '4' THEN 'Remove from cart'
                WHEN hits.eCommerceAction.action_type = '5' THEN 'Check out'
                WHEN hits.eCommerceAction.action_type = '6' THEN 'Purchase'
                WHEN hits.eCommerceAction.action_type = '7' THEN 'Refund'
                WHEN hits.eCommerceAction.action_type = '0' THEN 'Unknown'
            END AS Action
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits,
            UNNEST(hits.product) AS product
        WHERE
            _TABLE_SUFFIX BETWEEN '20160901' AND '20160930' AND
            hits.eCommerceAction.action_type = '6'
        GROUP BY Action, TrafficSource, Channel
        ORDER BY UserCount DESC
        LIMIT 20
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

           Channel  TotalRevenue  UserCount  AVGRevenuePerUser            TrafficSource    Action
0         Referral  561345860000       2584       2.172391e+08                 (direct)  Purchase
1   Organic Search  258471840000       1322       1.955158e+08                   google  Purchase
2           Direct  906368120000       1012       8.956207e+08                 (direct)  Purchase
3   Organic Search   77824060000        324       2.401977e+08                 (direct)  Purchase
4      Paid Search   31994760000        194       1.649214e+08                   google  Purchase
5      Paid Search   34812500000         94       3.703457e+08                 (direct)  Purchase
6          Display   17436460000         66       2.641888e+08                      dfa  Purchase
7         Referral    4728080000         56       8.443000e+07            dealspotr.com  Purchase
8         Referral   14532100000         42       3.460024e+08          mail.google.com  Purchase
9         Referral  

Referral searches is currently the most popular method when purchasing an item

Direct searches generates a greater revenue


In [144]:
# For each page title, find the number of customers and the way people are able to search for their products -> Which is the most effective
query = f"""
        SELECT
            IF(LENGTH(hits.page.pageTitle) <= 20, hits.page.pageTitle, CONCAT(SUBSTR(hits.page.pageTitle, 1, 20), '...')) AS PageTitle,
            channelGrouping AS Channel,
            COUNT(fullVisitorID) AS VisitorCount,
            SUM(IF(totals.transactions IS NULL, 0, totals.transactions)) AS TransactionCount
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`, UNNEST(hits) AS hits
        WHERE hits.page.pageTitle NOT LIKE '%Checkout%'
            AND hits.page.pageTitle NOT LIKE '%Home%'
            AND hits.page.pageTitle NOT LIKE '%Cart%'
            AND hits.page.pageTitle NOT LIKE '%Google%'
            AND hits.page.pageTitle NOT LIKE '%Payment%'
            AND hits.page.pageTitle NOT LIKE '%results%'
        GROUP BY PageTitle, Channel
        ORDER BY VisitorCount DESC
        LIMIT 20 
        """

rows = client.query(query)

result_df = rows.to_dataframe()

print(result_df, "\n")

           PageTitle         Channel  VisitorCount  TransactionCount
0     Men's-T-Shirts  Organic Search         31922              2698
1             Office  Organic Search         22441              2107
2   Page Unavailable        Referral         21349               499
3               Bags  Organic Search         19002              1288
4        Electronics  Organic Search         18657              2486
5          Drinkware  Organic Search         17328              1861
6     Men's-T-Shirts        Referral         14478              3573
7            Apparel  Organic Search         12960               899
8          Lifestyle  Organic Search         12844              1428
9               Bags        Referral         11839              1474
10            Office        Referral         11747              2691
11           YouTube  Organic Search         11369               472
12       Electronics        Referral         11298              1629
13         Drinkware        Referr

Referral and organic searches are clearly the most popular traffic channels and a higher traffic generally equates to a higher transaction count
